### checkking for ollama

In [ ]:
from langchain_community.llms import Ollama

llm = Ollama(model="gemma2:2b")

response = llm.invoke("Why won't i find someone?")

In [ ]:
print(response)

### checking for whisper

In [ ]:
import whisper

model = whisper.load_model("base")

# load audio and pad/trim it to fit 30 seconds
audio = whisper.load_audio("oh_hello_deadpool.mp3")
audio = whisper.pad_or_trim(audio)

# make log-Mel spectrogram and move to the same device as the model
mel = whisper.log_mel_spectrogram(audio).to(model.device)

# detect the spoken language
_, probs = model.detect_language(mel)
print(f"Detected language: {max(probs, key=probs.get)}")

# decode the audio
options = whisper.DecodingOptions()
result = whisper.decode(model, mel, options)

# print the recognized text
print(result.text)

### main code

In [ ]:
!pip install whisper ollama google-auth-oauthlib google-auth-httplib2 google-api-python-client

### basic working test

In [ ]:
import whisper
import ollama
import json
import os

def transcribe_audio(audio_file):
    model = whisper.load_model("base")
    
    # load audio and pad/trim it to fit 30 seconds
    audio = whisper.load_audio(audio_file)
    audio = whisper.pad_or_trim(audio)
    
    # make log-Mel spectrogram and move to the same device as the model
    mel = whisper.log_mel_spectrogram(audio).to(model.device)
    
    # detect the spoken language
    _, probs = model.detect_language(mel)
    detected_language = max(probs, key=probs.get)
    print(f"Detected language: {detected_language}")
    
    # decode the audio
    options = whisper.DecodingOptions()
    result = whisper.decode(model, mel, options)
    
    return result.text, detected_language

def process_with_ollama(text, language):
    # You can customize this prompt based on your needs
    prompt = f"""
    Language: {language}
    User input: {text}
    
    Analyze the user's input and determine the appropriate action or response. 
    If it's a command or question, provide a suitable answer or action plan.
    If it's a general statement, engage in a contextually appropriate conversation.
    """
    
    response = ollama.chat(model='gemma2:2b', messages=[
        {
            'role': 'system',
            'content': 'You are a helpful voice assistant.'
        },
        {
            'role': 'user',
            'content': prompt
        }
    ])
    
    return response['message']['content']

def text_to_speech(text):
    # Placeholder for text-to-speech functionality
    # You can integrate a TTS library or service here
    print("Text-to-speech output:", text)

def main():
    audio_file = "oh_hello_deadpool.mp3"
    
    # Step 1: Transcribe audio
    transcribed_text, detected_language = transcribe_audio(audio_file)
    print("Transcribed text:", transcribed_text)
    
    # Step 2: Process with Ollama
    ollama_response = process_with_ollama(transcribed_text, detected_language)
    print("Ollama response:", ollama_response)
    
    # Step 3: Convert response to speech
    text_to_speech(ollama_response)

if __name__ == "__main__":
    main()

### checking calendar api

In [2]:
!pip install google-api-python-client oauth2client

In [4]:
from __future__ import print_function
import datetime
import os.path
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

# If modifying these scopes, delete the file token.json.
SCOPES = ['https://www.googleapis.com/auth/calendar']

def get_credentials():
    creds = None
    # The file token.json stores the user's access and refresh tokens, and is
    # created automatically when the authorization flow completes for the first time.
    if os.path.exists('token.json'):
        creds = Credentials.from_authorized_user_file('token.json', SCOPES)
    # If there are no (valid) credentials available, let the user log in.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'client_secret.json', SCOPES)
            creds = flow.run_local_server(port=0)
        # Save the credentials for the next run
        with open('token.json', 'w') as token:
            token.write(creds.to_json())
    return creds

def create_event(service, event_details):
    event = {
        'summary': event_details['summary'],
        'location': event_details.get('location', ''),
        'description': event_details.get('description', ''),
        'start': {
            'dateTime': event_details['start'],
            'timeZone': 'Europe/Oslo',
        },
        'end': {
            'dateTime': event_details['end'],
            'timeZone': 'Europe/Oslo',
        },
    }

    event = service.events().insert(calendarId='primary', body=event).execute()
    print(f'Event created: {event.get("htmlLink")}')

def main():
    creds = get_credentials()
    try:
        service = build('calendar', 'v3', credentials=creds)

        # Example event details
        event_details = {
            'summary': 'Test Event',
            'location': 'Oslo, Norway',
            'description': 'This is a test event created by Python',
            'start': '2023-08-10T09:00:00',
            'end': '2023-08-10T10:00:00'
        }

        create_event(service, event_details)

    except HttpError as error:
        print(f'An error occurred: {error}')

if __name__ == '__main__':
    main()

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=638776883203-f5v5tb66cn48l8c6jd1ai7m5dmev08d5.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A58918%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcalendar&state=NIqfjrXrxLQzbbSTNsSx2dOVEuxTZT&access_type=offline


### test 2

In [ ]:
!pip install whisper ollama google-auth-oauthlib google-auth-httplib2 google-api-python-client

In [ ]:
!pip install pytz

In [ ]:
import whisper
import ollama
from datetime import datetime
import pytz
from google.oauth2.credentials import Credentials
from googleapiclient.discovery import build
from google.auth.transport.requests import Request
from google_auth_oauthlib.flow import InstalledAppFlow

SCOPES = ['https://www.googleapis.com/auth/calendar']

def transcribe_audio(audio_file):
    if not os.path.exists(audio_file):
        raise FileNotFoundError(f"The audio file '{audio_file}' does not exist.")
    
    try:
        model = whisper.load_model("base")
        audio = whisper.load_audio(audio_file)
        audio = whisper.pad_or_trim(audio)
        mel = whisper.log_mel_spectrogram(audio).to(model.device)
        options = whisper.DecodingOptions()
        result = whisper.decode(model, mel, options)
        return result.text
    except Exception as e:
        raise RuntimeError(f"Failed to transcribe audio: {str(e)}")


def natural_language_understanding(text):
    prompt = f"""
    Analyze the following user input and extract information for creating a calendar event:
    User input: {text}

    Please provide the following details in JSON format:
    1. Event title
    2. Start date and time
    3. End date and time (if specified, otherwise assume 1 hour duration)
    4. Location (if specified)
    5. Description (if any additional details are provided)

    If the input doesn't contain enough information for a calendar event, respond with {{"error": "Not enough information"}}.
    """
    
    response = ollama.chat(model='gemma2:2b', messages=[
        {
            'role': 'system',
            'content': 'You are a helpful assistant that extracts calendar event information from user input.'
        },
        {
            'role': 'user',
            'content': prompt
        }
    ])
    
    return response['message']['content']

def create_calendar_event(event_details):
    print("started creating calendar event")
    creds = None
    if creds and creds.expired and creds.refresh_token:
        creds.refresh(Request())
    else:
        flow = InstalledAppFlow.from_client_secrets_file('client_secret.json', SCOPES)
        creds = flow.run_local_server(port=0)
    
    service = build('calendar', 'v3', credentials=creds)
    
    event = {
        'summary': event_details['title'],
        'location': event_details.get('location', ''),
        'description': event_details.get('description', ''),
        'start': {
            'dateTime': event_details['start'],
            'timeZone': 'Europe/Oslos',
        },
        'end': {
            'dateTime': event_details['end'],
            'timeZone': 'Europe/Oslos',
        },
    }
    print("almost created calendar event")

    event = service.events().insert(calendarId='primary', body=event).execute()
    return f"Event created: {event.get('htmlLink')}"

def text_to_speech(text):
    # Placeholder for text-to-speech functionality
    print("Assistant:", text)

def main():
    audio_file = "recordings\meet_at_8.mp3"
    
    try:
        # Step 1: Speech-to-Text
        transcribed_text = transcribe_audio(audio_file)
        print("Transcribed text:", transcribed_text)
        
        # Step 2: Natural Language Understanding
        event_info_str = natural_language_understanding(transcribed_text)
        
        # Step 3: Intent Classification and Action
        try:
            event_info = json.loads(event_info_str)
            if "error" in event_info:
                response = "I'm sorry, but I couldn't extract enough information to create a calendar event. Could you please provide more details?"
            else:
                result = create_calendar_event(event_info)
                response = f"I've created the event for you. {result}"
        except json.JSONDecodeError as e:
            print(f"Error parsing JSON: {str(e)}")
            print(f"Raw response from LLM: {event_info_str}")
            response = "I'm sorry, but I encountered an error while processing your request. Could you please try rephrasing your input?"
        
        # Step 4: Text-to-Speech
        text_to_speech(response)
    
    except FileNotFoundError as e:
        print(f"Error: {str(e)}")
        print("Please make sure the audio file exists and the path is correct.")
    except RuntimeError as e:
        print(f"Error: {str(e)}")
        print("There was an issue processing the audio. Please try again or use a different audio file.")
    except Exception as e:
        print(f"An unexpected error occurred: {str(e)}")
        print("Please try again or contact support if the issue persists.")

if __name__ == "__main__":
    main()

In [3]:
import json
from datetime import datetime
import pytz
from google.oauth2.credentials import Credentials
from googleapiclient.discovery import build
from google.auth.transport.requests import Request
from google_auth_oauthlib.flow import InstalledAppFlow
from google_auth_oauthlib.flow import Flow

# Define the scope for Google Calendar API
SCOPES = ['https://www.googleapis.com/auth/calendar']

# def create_calendar_event(event_details):
#     print("Started creating calendar event")
    
#     creds = None
#     # Check if there are existing credentials
#     if creds and creds.expired and creds.refresh_token:
#         creds.refresh(Request())
#     else:
#         flow = InstalledAppFlow.from_client_secrets_file('client_secret.json', SCOPES)
#         creds = flow.run_local_server(port=0)
    
#     service = build('calendar', 'v3', credentials=creds)
    
#     event = {
#         'summary': event_details['title'],
#         'location': event_details.get('location', ''),
#         'description': event_details.get('description', ''),
#         'start': {
#             'dateTime': event_details['start'],
#             'timeZone': 'Europe/Oslo',
#         },
#         'end': {
#             'dateTime': event_details['end'],
#             'timeZone': 'Europe/Oslo',
#         },
#     }
    
#     print("Almost created calendar event")

#     event = service.events().insert(calendarId='primary', body=event).execute()
#     return f"Event created: {event.get('htmlLink')}"

def create_calendar_event(event_details):
    print("Started creating calendar event")
    
    creds = None
    # Check if there are existing credentials
    if creds and creds.expired and creds.refresh_token:
        creds.refresh(Request())
    else:
        flow = Flow.from_client_secrets_file(
            'client_secret.json',
            scopes=SCOPES,
            redirect_uri='urn:ietf:wg:oauth:2.0:oob')
        
        auth_url, _ = flow.authorization_url(prompt='consent')
        print(f'Please visit this URL to authorize the application: {auth_url}')
        code = input('Enter the authorization code: ')
        flow.fetch_token(code=code)
        creds = flow.credentials
    
    service = build('calendar', 'v3', credentials=creds)

def main():
    event_details_json = {
        "event": {
            "title": "Meeting with Rahul",
            "start_date_time": "2023-10-27T08:00:00",
            "end_date_time": "2023-10-27T09:00:00",
            "location": None,
            "description": "Meeting with Rahul at around 8"
        }
    }

    event_info = {
        'title': event_details_json['event']['title'],
        'start': event_details_json['event']['start_date_time'],
        'end': event_details_json['event']['end_date_time'],
        'location': event_details_json['event']['location'],
        'description': event_details_json['event']['description']
    }

    result = create_calendar_event(event_info)
    response = f"I've created the event for you. {result}"
    print(response)

if __name__ == "__main__":
    main()


Started creating calendar event
Please visit this URL to authorize the application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=638776883203-f5v5tb66cn48l8c6jd1ai7m5dmev08d5.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcalendar&state=945LOHw5Kt8c1tAVHatodw8PBM56GW&prompt=consent&access_type=offline
